# Analyzing Digital Media Sales with SQL

## Introduction

This project serves as a means for us to practice our SQL skills while answering business questions related to the sales of digital media from a fictional iTunes-like store. To do so, we analyze the contents of the [Chinook](https://github.com/lerocha/chinook-database) database, a sample database representing a digital media store and including tables containing information on employees, customers, purchases, artists, albums, etc. The full data dictionary for each table in the database is listed below:

- **`employee`**: table of employee information
    - `employee_id` -- employee identification number
    - `last_name` -- last name of employee
    - `first_name` -- first name of employee
    - `title` -- job title
    - `reports_to` -- identification number of employee's supervisor
    - `birthdate` -- employee date of birth
    - `hire_date` -- date employee was hired
    - `address` -- employee street address
    - `city` -- city of employee's address
    - `state` -- state of employee's address
    - `country` -- country of employee's address
    - `postal_code` -- zip code of employee's address
    - `phone` -- employee phone number
    - `fax` -- employee fax number
    - `email` -- employee email address
- **`customer`**: table of customer information
    - `customer_id` -- customer identification number
    - `first_name` -- customer's first name
    - `last_name` -- customer's last name
    - `company` -- company name of customer
    - `address` -- street address of customer
    - `city` -- city of customer's address
    - `state` -- state of customer's address
    - `country` -- country of customer's address
    - `postal_code` -- zip code of customer's address
    - `phone` -- customer phone number
    - `fax` -- customer fax number
    - `email` -- customer email address
    - `support_rep_id` -- employee identification number of sales support agent working with customer
- **`invoice`**: table of invoice information
    - `invoice_id` -- invoice identification number
    - `customer_id` -- customer identification number
    - `invoice_date` -- date of invoice
    - `billing_address` -- customer billing street address
    - `billing_city` -- city of customer billing address
    - `billing_state` -- state of customer billing address
    - `billing_country` -- country of customer billing address
    - `billing_postal_code` -- zip code of customer billing address
    - `total` -- invoice total
- **`invoice_line`**: table of individual invoice item information
    - `invoice_line_id` -- invoice line (for individual item/track purchased) identification number
    - `invoice_id` -- invoice identification number
    - `track_id` -- track identification number
    - `unit_price` -- price per unit
    - `quantity` -- number of units purchased 
- **`track`**: table of track information
    - `track_id` -- track identification number
    - `name` -- track name
    - `album_id` -- album identification number
    - `media_type_id` -- media type identification number
    - `genre_id` -- genre identification number
    - `composer` -- track composer
    - `milliseconds` -- track length in milliseconds
    - `bytes` -- track size in bytes
    - `unit_price` -- track price
- **`playlist_track`**: table of playlist track information
    - `playlist_id` -- playlist identification number
    - `track_id` -- track identification number
- **`playlist`**: table of playlist information
    - `playlist_id` -- playlist identification number
    - `name` -- playlist name
- **`album`**: table of album information
    - `album_id` -- album identification number
    - `title` -- album title
    - `artist_id` -- artist identification number
- **`artist`**: table of artist information
    - `artist_id` -- artist identification number
    - `name` -- artist name
- **`media_type`**: table of media type information
    - `media_type_id` -- media type identification number
    - `name` -- media type name
- **`genre`**: table of genre information
    - `genre_id` -- genre identification number
    - `name` -- genre name
    
The tables in the Chinook database are connected to each other through various shared columns. Our goal in this project is to practice writing complex queries that join multiple tables together in order to analyze the sales data of the Chinook digital media store and make recommendations for the company's business decisions based on that analysis.

## Exploring the Chinook Data

We begin by exploring the Chinook database to familiarize ourselves with the data it contains. First, we query the database to get a list of all its table and views. Then, we query a few of the tables to get familiar with their structures and contents.

In [56]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [57]:
%%sql
SELECT name, type
  FROM sqlite_master
 WHERE type IN ('table', 'view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [58]:
%%sql
SELECT *
  FROM album
 LIMIT 10;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7
10,Audioslave,8


In [59]:
%%sql
SELECT *
  FROM artist
 LIMIT 10;

Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains
6,Antônio Carlos Jobim
7,Apocalyptica
8,Audioslave
9,BackBeat
10,Billy Cobham


In [60]:
%%sql
SELECT *
  FROM invoice_line
 LIMIT 10;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


In [61]:
%%sql
SELECT *
  FROM track
 LIMIT 10;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99


## Identifying the Top Selling Genres in the US

Now that we're familiar with our data, we're ready to dive into our analysis. 

Our first task is to select the first three albums that should be added to the store after a deal signed with a new record label. The record label, which specializes in American artists and is paying Chinook to advertise the new albums in the United States, has put forth four potential albums by new artists, none of which have any tracks currently in the Chinook store, from which we can choose. We know only the artist name and the musical genre of each. We can make a recommendation for which three albums are predicted to have the highest earning potential and thus should be purchased for the store based on the musical genres that sell the most in the USA.

To identify the top selling genres in the US, we write a query below that returns the number of tracks sold in the USA per genre.

In [62]:
%%sql
-- select tracks sold in USA
WITH usa_tracks AS (
     SELECT il.*
       FROM invoice_line il
            INNER JOIN invoice i ON il.invoice_id = i.invoice_id
      WHERE i.billing_country = 'USA'
     )
-- count USA purchased tracks by genre
SELECT g.name genre_name,
       SUM(ut.quantity) tracks_purchased,
       SUM(ut.quantity) / CAST((SELECT COUNT(*) 
                                  FROM usa_tracks) 
                            AS FLOAT) percentage_tracks_purchased
  FROM usa_tracks ut
       INNER JOIN track t ON ut.track_id = t.track_id
       INNER JOIN genre g ON t.genre_id = g.genre_id
 GROUP BY g.genre_id
 ORDER BY tracks_purchased DESC;

Done.


genre_name,tracks_purchased,percentage_tracks_purchased
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Our query results show that Rock is the top selling genre in the USA, accounting for over 50% of all sales. The next top selling genres are Alternative & Punk and Metal, each with about 12% of sales. The remaining genres each make up a few percent or less of the total sales in America. 

The four albums being considered from the new record label are in the Hip-Hop, Punk, Pop, and Blues genres. Hip-Hop is ranked ninth in US sales per genre, Punk is ranked second, Pop is ranked eigth, and Blues is ranked fifth. Therefore, based solely on the top selling genres, we recommend adding the albums "Red Tone" (Punk), "Meteor and the Girls" (Pop), and "Slim Jim Bites" (Blues) to the store, as they are the most likely genres to sell.

## Analyzing Sales Support Agent Performance

Our next task is to analyze the performance of each sales support agent handling customer purchases. We would like to know which agents are performing the best and the worst, corresponding to having the largest and smallest dollar amounts of sales, and if there are any visible patterns between sales performance and other employee attributes. We can determine the sales agent performance by writing a query to find the total amount of sales in dollars assigned to each sales support agent within the company, as done below.

In [63]:
%%sql
-- calculate total sales per sales support agent
SELECT e.first_name || ' ' || e.last_name employee_name,
       ROUND(SUM(i.total), 2) sales_total,
       e.hire_date
  FROM invoice i
       INNER JOIN customer c ON i.customer_id = c.customer_id
       INNER JOIN employee e ON c.support_rep_id = e.employee_id
 GROUP BY e.employee_id
 ORDER BY sales_total DESC;

Done.


employee_name,sales_total,hire_date
Jane Peacock,1731.51,2017-04-01 00:00:00
Margaret Park,1584.0,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


The three sales support agents and their total sales are listed above: Jane Peacock is the top performing agent, with over \$1700 in sales, followed by Margaret Park, with nearly \$1600 in sales, and then by Steve Johnson, with almost \$1400 in sales. The difference in sales between the highest and lowest performing agents is approximately 20%, which roughly coincides with the difference in employment length between agents (~15%), as Jane (the top performer) was hired six and a half months before Steve (the bottom performer). There do not appear to be any other correlating factors to sales support agent performance.

## Analyzing Sales Data by Country

We then analyze the customer sales data for each different country, namely the total number of customers, the total number of sales, the total number of orders, the average value of sales per customers, and the average value of sales per order. In our following query, countries in the database with only one customer are collected into a group labeled "Other".

In [64]:
%%sql
-- create table with customers, sales, and orders per country
WITH country_info AS (
     SELECT cc.country,
            cc.customers,
            ci.sales,
            ci.orders
       -- get customer count per country
       FROM (SELECT country,
                    COUNT(*) customers
               FROM customer
              GROUP BY country) cc
                    -- get total sales, orders per country
                    INNER JOIN (SELECT c.country, 
                                       SUM(i.total) sales,
                                       COUNT(i.invoice_id) orders
                                  FROM customer c
                                       INNER JOIN invoice i 
                                               ON c.customer_id = i.customer_id
                                 GROUP BY country) ci
                            ON cc.country = ci.country
     )

-- collect one-customer countries into 'Other' category and calculate sales data
SELECT country_name,
       total_customers,
       total_sales,
       total_orders,
       avg_sales_per_customer,
       avg_sales_per_order
  FROM (SELECT CASE WHEN ci.customers = 1 THEN 'Other'
                    ELSE ci.country
                    END AS country_name,
               SUM(ci.customers) total_customers,
               ROUND(SUM(ci.sales), 2) total_sales,
               SUM(ci.orders) total_orders,
               ROUND(SUM(ci.sales) / sum(ci.customers), 2) avg_sales_per_customer,
               ROUND(SUM(ci.sales) / sum(ci.orders), 2) avg_sales_per_order
          FROM country_info ci
         GROUP BY country_name)
 -- sort in descending order by total sales; place 'Other' last
 ORDER BY CASE WHEN country_name = 'Other' THEN 1
               ELSE 0
               END ASC,
          total_sales DESC;


Done.


country_name,total_customers,total_sales,total_orders,avg_sales_per_customer,avg_sales_per_order
USA,13,1040.49,131,80.04,7.94
Canada,8,535.59,76,66.95,7.05
Brazil,5,427.68,61,85.54,7.01
France,5,389.07,50,77.81,7.78
Germany,4,334.62,41,83.66,8.16
Czech Republic,2,273.24,30,136.62,9.11
United Kingdom,3,245.52,28,81.84,8.77
Portugal,2,185.13,29,92.57,6.38
India,2,183.15,21,91.57,8.72
Other,15,1094.94,147,73.0,7.45


The US is the leading country in terms of number of customers, sales, and orders, but the average sales per customer and per order among US sales is on par with those among sales in other countries. The Czech Republic has the highest average sales per customer and per order, but there are low statistics for this and many other countries, so it's impossible to draw any real conclusion from this.

## Conclusion

In this project, we have analyzed sales data from the Chinook database, a sample database representing a fictional digital media store, using SQL. We have identified the top selling genres in the US market as Rock, Alternative & Punk, and Metal. We also analyzed the sales data by country, finding the USA to be the largest customer using the store. In the future, we will investigate the type of purchases customers make, comparing full album to single track purchases, to determine which purchases are more profitable.